In [5]:
# type this in cmd  < wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" >
import gensim, logging


In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
a= 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
b= 'GoogleNews-vectors-negative300.bin.gz'
gmodel = gensim.models.KeyedVectors.load_word2vec_format(b,binary=True)

2021-01-24 20:35:11,002 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin.gz
2021-01-24 20:35:50,623 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin.gz


In [8]:
gmodel['cat']

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [9]:
gmodel['spatula']

array([-0.19140625, -0.04296875,  0.27539062,  0.00488281, -0.3203125 ,
        0.08203125,  0.05566406, -0.03613281, -0.31445312,  0.10693359,
       -0.359375  ,  0.29882812,  0.02331543,  0.05517578, -0.140625  ,
        0.1953125 , -0.23632812, -0.22167969, -0.06542969, -0.3359375 ,
        0.25195312, -0.09326172,  0.54296875,  0.11328125, -0.28710938,
       -0.12011719, -0.11181641,  0.20996094, -0.33203125,  0.30273438,
       -0.3359375 , -0.12255859,  0.12890625, -0.28515625, -0.04223633,
        0.25585938,  0.3203125 ,  0.07177734,  0.19042969, -0.01379395,
        0.16992188, -0.22460938,  0.5078125 ,  0.08398438, -0.07519531,
       -0.06396484,  0.05371094,  0.34570312,  0.46289062, -0.16699219,
       -0.30664062,  0.15234375, -0.09765625, -0.26171875, -0.14160156,
        0.2265625 ,  0.49609375, -0.10791016, -0.08447266,  0.234375  ,
        0.04931641, -0.07128906,  0.05273438, -0.11914062,  0.09814453,
        0.11181641, -0.13574219, -0.46875   ,  0.26171875,  0.12

In [10]:
gmodel.similarity('cat','dog')

0.76094574

In [11]:
gmodel.similarity('rabbit','dog')

0.5868356

In [12]:
gmodel.similarity('frog','dog')

0.3550165

In [13]:
gmodel.similarity('frog','chair')

0.11644859

In [14]:
gmodel.similarity('cat','spatula')

0.12412613

In [15]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [16]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) # strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remove apostrophes
    sent = re.sub(r'\W', ' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated spaces
    sent = sent.strip()
    return sent.split()

In [17]:
import re
import os
unsup_sentences = []

# source: http://ai.stanford.edu/~amaas/data/sentiment/ data from IMDB

for dirname in ['train/pos','train/neg','train/unsup','test/pos','test/neg']:
    for fname in sorted(os.listdir('aclImdb/'+dirname)):
        if fname[-4:] == '.txt':
            with open('aclImdb/' + dirname +'/'+fname,encoding='UTF-8') as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname+'/'+fname]))

# source: http://www.cs.cornell.edu/people/pabo/movie-review-data/

for dirname in ['txt_sentoken/pos','txt_sentoken/neg']:
    for fname in sorted(os.listdir(dirname)):
        if fname[-4:] == '.txt':
            with open(dirname + '/' + fname, encoding='UTF-8') as f:
                for i,sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words,['%s/%s-%d' %(dirname,fname,i)]))

# source: https://nlp.stanford.edu/sentiment/, data from Rotten Tomatoes

with open('stanfordSentimentTreebank/original_rt_snippets.txt',encoding='UTF-8') as f:
    for i,line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocument(words,['rt-%d'% i]))

In [18]:
len(unsup_sentences)

175321

In [19]:
unsup_sentences[0:10]

tire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 'to', 'bromwell', 'high', 'i', 'expect', 'that', 'many', 'adults', 'of', 'my', 'age', 'think', 'that', 'bromwell', 'high', 'is', 'far', 'fetched', 'what', 'a', 'pity', 'that', 'it', 'is'], tags=['train/pos/0_9.txt']),
 TaggedDocument(words=['homelessness', 'or', 'houselessness', 'as', 'george', 'carlin', 'stated', 'has', 'be

In [20]:
import random

class PermuteSentences(object):
    def __init__(self,sents):
        self.sents = sents
    
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [21]:
permuter = PermuteSentences(unsup_sentences)
model = Doc2Vec(permuter,dm=0,hs=1,size=50)


0000, processed 11262154 words (4930018/s), 111135 word types, 80000 tags
2021-01-24 20:40:07,965 : INFO : PROGRESS: at example #90000, processed 12681119 words (4509386/s), 116790 word types, 90000 tags
2021-01-24 20:40:08,260 : INFO : PROGRESS: at example #100000, processed 14070490 words (4722741/s), 121864 word types, 100000 tags
2021-01-24 20:40:08,562 : INFO : PROGRESS: at example #110000, processed 15456291 words (4597954/s), 126783 word types, 110000 tags
2021-01-24 20:40:08,850 : INFO : PROGRESS: at example #120000, processed 16866466 words (4918099/s), 131630 word types, 120000 tags
2021-01-24 20:40:09,130 : INFO : PROGRESS: at example #130000, processed 18267205 words (5018584/s), 136435 word types, 130000 tags
2021-01-24 20:40:09,419 : INFO : PROGRESS: at example #140000, processed 19712303 words (5015115/s), 140872 word types, 140000 tags
2021-01-24 20:40:09,724 : INFO : PROGRESS: at example #150000, processed 21132997 words (4659809/s), 145114 word types, 150000 tags
2021

In [22]:
model.delete_temporary_training_data(keep_inference=True)

In [23]:
model.save('reviews.d2v')
# in other program, we could write: model = Doc2Vec.load('reviews.d2v)

2021-01-24 20:43:12,181 : INFO : saving Doc2Vec object under reviews.d2v, separately None
2021-01-24 20:43:13,978 : INFO : saved reviews.d2v


In [24]:
model.infer_vector(extract_words('This place is not worth your time, let alone Vegas.'))

array([ 0.15615003,  0.35533318,  0.0244548 ,  0.31123462,  0.07109641,
       -0.23480405,  0.21650085,  0.03178753,  0.28266463, -0.10115225,
        0.12566638, -0.21731445,  0.21311566, -0.30245453, -0.01227137,
       -0.2411961 , -0.23300245,  0.00300279,  0.11262076, -0.1246913 ,
        0.00953154, -0.15050216,  0.02852447, -0.33485866, -0.07655568,
       -0.04546369,  0.2395672 , -0.06979701, -0.02417124, -0.11960468,
        0.05165534, -0.10228276,  0.01345713,  0.23048018, -0.15954658,
       -0.07275184, -0.14637718,  0.5132226 ,  0.02547812, -0.03648876,
        0.04452513,  0.02096366,  0.26291052, -0.032118  ,  0.31133208,
       -0.1114691 ,  0.03198015,  0.24159124, -0.26068214, -0.05742063],
      dtype=float32)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [model.infer_vector(extract_words('This place is not worth your time, let alone Vegas.'))],
    [model.infer_vector(extract_words('Service sucks.'))]
)

array([[0.56397027]], dtype=float32)

In [26]:
cosine_similarity(
    [model.infer_vector(extract_words('Highly recommended.'))],
    [model.infer_vector(extract_words('Service sucks.'))]
)

array([[0.30007932]], dtype=float32)

In [27]:
sentences = []
sentvecs = []
sentiments = []
for fname in ['yelp','amazon_cells','imdb']:
    with open('sentiment labelled sentences/%s_labelled.txt'% fname, encoding='UTF-8') as f:
        for i,line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words,steps=10)) # create a vector for this document
            sentiments.append(int(line_split[1]))

#shuffle sentences, sentvecs, sentiments together 

combined = list(zip(sentences,sentvecs,sentiments))
random.shuffle(combined)
sentences,sentvecs,sentiments=zip(*combined)

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [29]:
scores = cross_val_score(clf,sentvecs, sentiments, cv=5)
np.mean(scores),np.std(scores)

(0.8029999999999999, 0.003858612300930068)

In [30]:
scores = cross_val_score(clfrf,sentvecs,sentiments,cv=5)
np.mean(scores),np.std(scores)

(0.7996666666666666, 0.01007747763855399)

In [31]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
pipeline = make_pipeline(CountVectorizer(),TfidfTransformer(),RandomForestClassifier())

In [32]:
 scores = cross_val_score(pipeline,sentences, sentiments,cv=5)
 np.mean(scores),np.std(scores)

(0.7813333333333333, 0.018720161442798634)